In [6]:
#Installs tensor flow well set as on-start scrip later
!pip install tensorflow

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
  Using cached tensorflow-2.8.0-cp37-cp37m-manylinux2010_x86_64.whl (497.5 MB)
  Using cached tf_estimator_nightly-2.8.0.dev2021122109-py2.py3-none-any.whl (462 kB)
  Using cached grpcio-1.44.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.3 MB)
  Using cached gast-0.5.3-py3-none-any.whl (19 kB)
  Using cached absl_py-1.0.0-py3-none-any.whl (126 kB)
  Using cached libclang-13.0.0-py2.py3-none-manylinux1_x86_64.whl (14.5 MB)
  Using cached keras-2.8.0-py2.py3-none-any.whl (1.4 MB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached termcolor-1.1.0-py3-

In [7]:
import os
import json
import sagemaker
import numpy as np
import pandas as pd
import tensorflow as tf
from sagemaker import get_execution_role
from sagemaker.tensorflow import TensorFlow

# Gets number of users and pages from stored data (data prep)
%store -r n_user
%store -r n_item
%store -r sites_id

# Get bucket/sagemaker session data
sage_session = sagemaker.Session()
role = get_execution_role()
bucket = sage_session.default_bucket()

# Location for training dataset
training_uri = os.path.join(f's3://{bucket}', 'data')

# Sagemaker instance data subject to change after more research
num_of_instance = 1
instance_type = 'ml.c5.xlarge'
train_script = "script.py"
#ml.c5.xlarge
# Some base parameters too be tweaked later
training_parameters = {
    'epochs': 1,
    'batch_size': 256,
    'n_user': n_user,
    'n_item': n_item
}

# training framework specs
tensorflow_version = '2.1.0'
python_version = 'py3'
distributed_training_spec = {'parameter_server': {'enabled': True}}

# Instantiate training job
ncf_estimator = TensorFlow(
    entry_point=train_script,
    role=role,
    train_instance_count=num_of_instance,
    train_instance_type=instance_type,
    framework_version=tensorflow_version,
    py_version=python_version,
    distributions=distributed_training_spec,
    hyperparameters=training_parameters
)

# Start training
ncf_estimator.fit(training_uri)

# Deploying of training model so it can be called
endpoint_name = 'odb-recommendation-engine'

ncf_estimator.deploy(initial_instance_count=1,
                        instance_type='ml.t2.medium',
                        endpoint_name=endpoint_name)


distributions has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


2022-03-28 18:34:43 Starting - Starting the training job...
2022-03-28 18:35:08 Starting - Preparing the instances for trainingProfilerReport-1648492483: InProgress
......
2022-03-28 18:36:12 Downloading - Downloading input data...
2022-03-28 18:36:33 Training - Downloading the training image.....2022-03-28 18:37:19,714 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2022-03-28 18:37:19,723 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2022-03-28 18:37:20,038 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2022-03-28 18:37:20,062 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2022-03-28 18:37:20,086 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2022-03-28 18:37:20,101 sagemaker-containers INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {
        "sagemaker_parameter_server_enabled": true
  

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Training seconds: 89
Billable seconds: 89
----------!

In [20]:
# Calling locally for testing
from sagemaker.tensorflow import TensorFlowPredictor

predictor = TensorFlowPredictor('odb-recommendation-engine')

# load testing data
base_dir = os.path.join(f's3://{bucket}', 'data')
df_test = np.load(os.path.join("", 'test.npy'), allow_pickle=True)
df_test = np.asarray(df_test).astype(int)
user_test, item_test, y_test = np.split(np.transpose(df_test).flatten(), 3)



test_user_data = np.ndarray.tolist(np.asarray(tf.one_hot(user_test, depth=n_user)))
test_item_data = np.ndarray.tolist(np.asarray(tf.one_hot(item_test, depth=n_item)))
    
batch_size = 100
y_pred = []
for idx in range(0, len(test_user_data), batch_size):
    # reformat test samples into tensorflow serving acceptable format
    input_vals = {
     "instances": [
         {'input_1': u, 'input_2': i} 
         for (u, i) in zip(test_user_data[idx:idx+batch_size], test_item_data[idx:idx+batch_size])
    ]}
 
    # invoke model endpoint to make inference
    pred = predictor.predict(input_vals)
    
    # store predictions
    y_pred.extend([i[0] for i in pred['predictions']])
    
pred_df = pd.DataFrame([
    user_test,
    item_test,
    (np.array(y_pred)).astype(float)],
).T


pred_df.columns = ['userId', 'site', 'accuracy']


print(pred_df.head(), end='\n\n\n')


   userId          site  prediction
0   115.0  9.130000e+18    0.506187
1    32.0  1.090000e+18    0.505965
2    72.0  7.900000e+18    0.506877
3   151.0  7.670000e+17    0.507576
4    67.0  8.240000e+18    0.505972




In [4]:
# Using this on my free account to delete endpoint after testing to avoid unnecesary costs
predictor.delete_endpoint(delete_endpoint_config=True)